# Libs

In [ ]:
import json
import pandas as pd
import numpy as np
import nltk
import re
nltk.download('stopwords')
nltk.download('rslp')
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
stop_words = set(stopwords.words('portuguese'))
stemmer = RSLPStemmer()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.combine import SMOTEENN, SMOTETomek
from xgboost import XGBClassifier
from collections import Counter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


### Importar a base

In [ ]:
# Função para carregar arquivos JSON
def carregar_json(caminho):
    with open(caminho, 'r', encoding='utf-8') as f:
        return json.load(f)

# Carregar os arquivos
jobs = carregar_json('vagas.json')
prospects = carregar_json('prospects.json')
applicants = carregar_json('applicants.json')

# Converter jobs.json em DataFrame
df_jobs = pd.DataFrame.from_dict(jobs, orient='index').reset_index().rename(columns={'index': 'job_id'})

# Converter applicants.json em DataFrame
df_applicants = pd.DataFrame.from_dict(applicants, orient='index').reset_index().rename(columns={'index': 'applicant_id'})

# Tratar estrutura variada do prospects.json
lista_prospects = []

for job_id, value in prospects.items():
    if isinstance(value, list):
        for candidato in value:
            if isinstance(candidato, dict):
                lista_prospects.append({'job_id': job_id, **candidato})
    elif isinstance(value, dict) and 'prospects' in value:
        for candidato in value['prospects']:
            if isinstance(candidato, dict):
                lista_prospects.append({'job_id': job_id, **candidato})
    else:
        print(f"[AVISO] Estrutura inesperada na vaga {job_id}: {type(value)}")

df_prospects = pd.DataFrame(lista_prospects)

In [ ]:
# Unir os dados: prospects + applicants + jobs
df_merge = df_prospects.merge(df_applicants, left_on='codigo', right_on='applicant_id', how='left')
df_merge = df_merge.merge(df_jobs, on='job_id', how='left')
print("Formato final da base:", df_merge.shape)
display(df_merge.head())


Formato final da base: (53759, 19)


,job_id,nome,codigo,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador,applicant_id,infos_basicas,informacoes_pessoais,informacoes_profissionais,formacao_e_idiomas,cargo_atual,cv_pt,cv_en,informacoes_basicas,perfil_vaga,beneficios
0,4530,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira,25632,"{'telefone_recado': '', 'telefone': '(21) 9348...",{'data_aceite': 'Cadastro anterior ao registro...,"{'titulo_profissional': '', 'area_atuacao': ''...","{'nivel_academico': '', 'nivel_ingles': '', 'n...",{},\ndados pessoais\nestado civil: casado\nidade:...,,"{'data_requicisao': '10-03-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'Rio de Janeiro',...","{'valor_venda': '-', 'valor_compra_1': 'R$', '..."
1,4530,Srta. Isabela Cavalcante,25529,Encaminhado ao Requisitante,22-03-2021,23-03-2021,"encaminhado para - R$ 6.000,00 – CLT Full , n...",Ana Lívia Moreira,25529,"{'telefone_recado': '', 'telefone': '(21) 9426...",{'data_aceite': 'Cadastro anterior ao registro...,{'titulo_profissional': 'ANALISTA DE REDES E T...,{'nivel_academico': 'Ensino Superior Completo'...,{},"solteiro, 47 anos\n\nestrada meringuava, nº 17...",,"{'data_requicisao': '10-03-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'Rio de Janeiro',...","{'valor_venda': '-', 'valor_compra_1': 'R$', '..."
2,4531,Sra. Yasmin Fernandes,25364,Contratado pela Decision,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano,25364,"{'telefone_recado': '', 'telefone': '(21) 9633...",{'data_aceite': 'Cadastro anterior ao registro...,{'titulo_profissional': 'Liderança / Desenvolv...,{'nivel_academico': 'Ensino Superior Completo'...,{},\n\nárea de atuação: lider de consultoria / ge...,,"{'data_requicisao': '10-03-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'hora',..."
3,4531,Alexia Barbosa,25360,Encaminhado ao Requisitante,17-03-2021,17-03-2021,,Juliana Cassiano,25360,"{'telefone_recado': '', 'telefone': '(11) 9774...",{'data_aceite': 'Cadastro anterior ao registro...,"{'titulo_profissional': '', 'area_atuacao': ''...","{'nivel_academico': '', 'nivel_ingles': '', 'n...",{},informações pessoais\n estado civil: casado\n...,,"{'data_requicisao': '10-03-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'hora',..."
4,4533,Arthur Almeida,26338,Contratado pela Decision,29-04-2021,18-05-2021,,Stella Vieira,26338,"{'telefone_recado': '', 'telefone': '(31) 9270...","{'data_aceite': '25/11/2022 11:04', 'nome': 'A...","{'titulo_profissional': '', 'area_atuacao': ''...","{'nivel_academico': '', 'nivel_ingles': '', 'n...",{},"solteiro, brasileiro, 21/06/1987\nhabilitação ...",,"{'data_requicisao': '11-03-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '207,00 -', 'valor_compra_1': ..."


In [ ]:
df_merge['situacao_candidado'].value_counts()

,count
situacao_candidado,
Prospect,20021
Encaminhado ao Requisitante,16122
Inscrito,3980
Não Aprovado pelo Cliente,3492
Contratado pela Decision,2758
Desistiu,2349
Não Aprovado pelo RH,1765
Não Aprovado pelo Requisitante,765
Entrevista Técnica,579


In [ ]:
# Categorias que podem indicar sucesso na contratação, mas aí tem que ver
categorias_contratado = [
    'Contratado pela Decision',
    'Contratado como Hunting',
    'Proposta Aceita'
]

# Criando a variável alvo
df_merge['target_contratado'] = df_merge['situacao_candidado'].apply(lambda x: 1 if x in categorias_contratado else 0)

# Preencher nulos com string vazia
df_modelo = df_merge.copy()
colunas_texto = ['formacao_e_idiomas', 'cargo_atual', 'perfil_vaga', 'beneficios','informacoes_profissionais', 'infos_basicas']
df_modelo[colunas_texto] = df_modelo[colunas_texto].fillna('')

# Criar coluna de texto consolidado
df_modelo['texto_geral'] = df_modelo[colunas_texto].astype(str).agg(' '.join, axis=1)

##  Estratégias de pré-processamento para comparar:

| Estratégia             | O que faz                                 |
| ---------------------- | ----------------------------------------- |
| `raw`                  | Sem pré-processamento, apenas `.lower()`  |
| `stopwords`            | Remove stopwords (NLTK)                   |
| `stopwords + stemming` | Remove stopwords e aplica stemmer (RSLP)  |
| `custom_cleaning`      | Remove pontuação, números, símbolos, etc. |


In [ ]:
# Funções diferentes de pré-processamento
def raw(texto):
    return texto.lower() if isinstance(texto, str) else ''

def clean_stopwords(texto):
    if not isinstance(texto, str): return ''
    tokens = re.sub(r'[\W\d_]+', ' ', texto.lower()).split()
    return ' '.join([t for t in tokens if t not in stop_words and len(t) > 2])

def clean_stopwords_stem(texto):
    if not isinstance(texto, str): return ''
    tokens = re.sub(r'[\W\d_]+', ' ', texto.lower()).split()
    return ' '.join([stemmer.stem(t) for t in tokens if t not in stop_words and len(t) > 2])

# Dicionário com as funções
estrategias = {
    'raw': raw,
    'stopwords': clean_stopwords,
    'stopwords_stem': clean_stopwords_stem
}

# Dados de entrada
coluna_texto = 'cv_pt'
y = df_modelo['target_contratado']
X_texto_original = df_modelo[coluna_texto].fillna('')

# Divisão treino/teste
X_train_txt, X_test_txt, y_train, y_test = train_test_split(
    X_texto_original, y, stratify=y, test_size=0.3, random_state=42
)

# Loop de avaliação
resultados = {}

for nome, funcao in estrategias.items():
    print(f"\n Testando: {nome}")
    X_train_proc = X_train_txt.apply(funcao)
    X_test_proc = X_test_txt.apply(funcao)

    vectorizer = TfidfVectorizer(max_features=3000)
    X_train_vect = vectorizer.fit_transform(X_train_proc)
    X_test_vect = vectorizer.transform(X_test_proc)

    # Balanceamento
    sampler = SMOTETomek(random_state=42)
    X_res, y_res = sampler.fit_resample(X_train_vect, y_train)

    # Modelo
    modelo = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=1995)
    modelo.fit(X_res, y_res)

    # Avaliação no teste
    y_pred = modelo.predict(X_test_vect)
    relatorio = classification_report(y_test, y_pred, output_dict=True)

    # Guardar métricas
    resultados[nome] = {
        'precision': relatorio['1']['precision'],
        'recall': relatorio['1']['recall'],
        'f1_score': relatorio['1']['f1-score']
    }


pd.DataFrame(resultados).T.sort_values(by='f1_score', ascending=False)


 Testando: raw

 Testando: stopwords

 Testando: stopwords_stem


,precision,recall,f1_score
stopwords_stem,0.086337,0.486607,0.146653
raw,0.084563,0.524554,0.145646
stopwords,0.085271,0.466518,0.144188


In [ ]:
# Função de pré-processamento final (stopwords + stem)
def clean_stopwords_stem(texto):
    if not isinstance(texto, str): return ''
    tokens = re.sub(r'[\W\d_]+', ' ', texto.lower()).split()
    return ' '.join([stemmer.stem(t) for t in tokens if t not in stop_words and len(t) > 2])

# Combinar múltiplas colunas em uma
colunas_texto = ['cv_pt', 'informacoes_profissionais', 'formacao_e_idiomas', 'perfil_vaga', 'beneficios']
df_modelo['texto_completo'] = df_modelo[colunas_texto].fillna('').astype(str).agg(' '.join, axis=1)
df_modelo['texto_completo_limpo'] = df_modelo['texto_completo'].apply(clean_stopwords_stem)

# Separar treino/teste
X_texto = df_modelo['texto_completo_limpo'].fillna('')
y = df_modelo['target_contratado']
X_train_txt, X_test_txt, y_train, y_test = train_test_split(X_texto, y, stratify=y, test_size=0.3, random_state=1995)

# Vetorizar texto
vectorizer = TfidfVectorizer(max_features=3000)
X_train_vect = vectorizer.fit_transform(X_train_txt)
X_test_vect = vectorizer.transform(X_test_txt)


## Estratégias para o desbalanceamento



### Estratégias que vamos comparar:

| **Técnica**                | **Descrição**                                |
| ---------------------- | ---------------------------------------- |
| **SMOTE**              | Oversampling gerando exemplos sintéticos |
| **RandomOverSampler**  | Repete exemplos da classe minoritária    |
| **RandomUnderSampler** | Remove exemplos da classe majoritária    |
| **SMOTEENN**           | SMOTE + edição de instâncias com ENN     |
| **SMOTETomek**         | SMOTE + remoção de pares Tomek (ruídos)  |


In [ ]:
estrategias = {
    'RandomOverSampler': RandomOverSampler(random_state=1995),
    'RandomUnderSampler': RandomUnderSampler(random_state=1995),
    'SMOTE': SMOTE(random_state=1995),
    'SMOTEENN': SMOTEENN(random_state=1995),
    'SMOTETomek': SMOTETomek(random_state=1995)
}

# Função de pré-processamento (stopwords + stem)
def clean_stopwords_stem(texto):
    if not isinstance(texto, str): return ''
    tokens = re.sub(r'[\W\d_]+', ' ', texto.lower()).split()
    return ' '.join([stemmer.stem(t) for t in tokens if t not in stop_words and len(t) > 2])

# Juntar todas as colunas de texto
colunas_texto = ['cv_pt', 'informacoes_profissionais', 'formacao_e_idiomas', 'perfil_vaga', 'beneficios']
df_modelo['texto_completo'] = df_modelo[colunas_texto].fillna('').astype(str).agg(' '.join, axis=1)
df_modelo['texto_completo_limpo'] = df_modelo['texto_completo'].apply(clean_stopwords_stem)

# Separar variáveis
X_texto = df_modelo['texto_completo_limpo']
y = df_modelo['target_contratado']

# Split treino/teste
X_train_txt, X_test_txt, y_train, y_test = train_test_split(X_texto, y, stratify=y, test_size=0.3, random_state=1995)

# Vetorização
vectorizer = TfidfVectorizer(max_features=3000)
X_train_vect = vectorizer.fit_transform(X_train_txt)
X_test_vect = vectorizer.transform(X_test_txt)

# Avaliação por estratégia
resultados = []

for nome, sampler in estrategias.items():
    print(f"\n Estratégia de balanceamento: {nome}")

    X_res, y_res = sampler.fit_resample(X_train_vect, y_train)

    modelo = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=1995)
    modelo.fit(X_res, y_res)

    y_pred = modelo.predict(X_test_vect)
    relatorio = classification_report(y_test, y_pred, output_dict=True)

    resultados.append({
        'estrategia': nome,
        'precision_0': relatorio['0']['precision'],
        'recall_0': relatorio['0']['recall'],
        'f1_0': relatorio['0']['f1-score'],
        'precision_1': relatorio['1']['precision'],
        'recall_1': relatorio['1']['recall'],
        'f1_1': relatorio['1']['f1-score']
    })

df_resultados = pd.DataFrame(resultados)
df_resultados.sort_values(by='f1_1', ascending=False)


 Estratégia de balanceamento: RandomOverSampler

 Estratégia de balanceamento: RandomUnderSampler

 Estratégia de balanceamento: SMOTE

 Estratégia de balanceamento: SMOTEENN

 Estratégia de balanceamento: SMOTETomek


,estrategia,precision_0,recall_0,f1_0,precision_1,recall_1,f1_1
2,SMOTE,0.970628,0.841780,0.901624,0.174092,0.566964,0.266387
4,SMOTETomek,0.970506,0.840336,0.900742,0.172508,0.565848,0.264407
0,RandomOverSampler,0.971608,0.824514,0.892038,0.165209,0.590402,0.258175
3,SMOTEENN,0.973969,0.749212,0.846933,0.133983,0.659598,0.222725
1,RandomUnderSampler,0.973157,0.752101,0.848467,0.133150,0.647321,0.220868


## Conclusões
* Melhor coluna de texto (texto_completo_limpo);
* Melhor pré-processamento (stopwords + stem);
* Melhor balanceamento (SMOTE);